In [1]:
from sklearn.datasets import *
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

/home/grads/aafiyahussain/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-19 21:26:54,320	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
sampling_params = SamplingParams(max_tokens=512,temperature=1.2)

In [3]:
model_id = "Qwen/Qwen2.5-14B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = LLM(
    model=model_id,
    tensor_parallel_size=1,
    max_model_len=8192
)

INFO 04-19 21:26:55 __init__.py:207] Automatically detected platform cuda.
INFO 04-19 21:27:04 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 04-19 21:27:04 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-14B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-14B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:00<00:01,  6.85it/s]
Loading safetensors checkpoint shards:  25% Completed | 2/8 [00:00<00:01,  3.62it/s]
Loading safetensors checkpoint shards:  38% Completed | 3/8 [00:00<00:01,  2.88it/s]
Loading safetensors checkpoint shards:  50% Completed | 4/8 [00:01<00:01,  2.61it/s]
Loading safetensors checkpoint shards:  62% Completed | 5/8 [00:01<00:01,  2.48it/s]
Loading safetensors checkpoint shards:  75% Completed | 6/8 [00:02<00:00,  2.42it/s]
Loading safetensors checkpoint shards:  88% Completed | 7/8 [00:02<00:00,  2.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:03<00:00,  2.36it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:03<00:00,  2.57it/s]



INFO 04-19 21:27:10 model_runner.py:1115] Loading model weights took 27.5681 GB
INFO 04-19 21:27:13 worker.py:267] Memory profiling takes 2.32 seconds
INFO 04-19 21:27:13 worker.py:267] the current vLLM instance can use total_gpu_memory (44.43GiB) x gpu_memory_utilization (0.90) = 39.99GiB
INFO 04-19 21:27:13 worker.py:267] model weights take 27.57GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.47GiB; the rest of the memory reserved for KV Cache is 10.90GiB.
INFO 04-19 21:27:13 executor_base.py:111] # cuda blocks: 3719, # CPU blocks: 1365
INFO 04-19 21:27:13 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 7.26x
INFO 04-19 21:27:16 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.03it/s]

INFO 04-19 21:27:33 model_runner.py:1562] Graph capturing finished in 17 secs, took 0.28 GiB
INFO 04-19 21:27:33 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 22.80 seconds


In [4]:
def get_prompt(summary_statistics,sampled_rows):
    prompt = f"""You are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  
    You will be provided with two inputs:

    1. **Summary statistics** (the comma seperated output of `df.describe()`)  
    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.

    Your task is to:

    1. Choose **one** visualization from this list of common exploratory plots:
    - Histogram  
    - Box plot  
    - Scatter plot  
    - Bar chart  
    - Line chart  
    - Heatmap (e.g. correlation matrix)  
    - Pair plot (scatterplot matrix)  
    - Violin plot  
    - Density plot  
    - Stacked bar chart  

    2. Explain your **rationale** for selecting that visualization given the summary statistics and sample rows.

    Inputs:
    1. Summary statistics:
    {summary_statistics}

    2. Ten randomly selected rows from the dataset:
    {sampled_rows}

    Provide your answer in the format \\boxed{{answer}} at the end."""
    return prompt

In [5]:
def get_summary_stat_and_rows(df):
    summary_statistics = df.describe().to_csv(index=True)
    sampled_rows = df.sample(n=10, random_state=42).to_csv(index=False)
    return summary_statistics,sampled_rows

## Iris

In [6]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = [iris.target_names[t] for t in iris.target]
# csv_data = df_iris.to_csv(index=False)

In [7]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_iris)

In [8]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [9]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [10]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs

[RequestOutput(request_id=0, prompt='<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nYou are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  \n    You will be provided with two inputs:\n\n    1. **Summary statistics** (the comma seperated output of `df.describe()`)  \n    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.\n\n    Your task is to:\n\n    1. Choose **one** visualization from this list of common exploratory plots:\n    - Histogram  \n    - Box plot  \n    - Scatter plot  \n    - Bar chart  \n    - Line chart  \n    - Heatmap (e.g. correlation matrix)  \n    - Pair plot (scatterplot matrix)  \n    - Violin plot  \n    - Density plot  \n    - Stacked bar chart  \n\n    2. Explain your **rationale** for selecting that visualization given the summary statistics and sample rows.\n\n    Inputs:\n    1. Summa

In [11]:
outputs[0].outputs[0].text #scatter plot

'Given the summary statistics and sample rows from the dataset, I would recommend a **scatter plot** for the exploratory data analysis in this instance.\n\n**Rationale:**\n\n1. **Summary Statistics:** The statistics reveal that the dataset consists of four continuous variables (sepal length, sepal width, petal length, petal width), each with a min, max, mean, and standard deviation (among others). Given these variables represent physical measurements, and knowing the target column seems associated with a category (e.g., "setosa", "versicolor", "virginica"), a scatter plot could be quite illuminating.\n\n2. **Sample Rows Observation:** The sample rows from the dataset include an additional column `"target"` that classifies the entries as species types ("setosa", "versicolor", "virginica"). A scatter plot would be an effective way to visualize the distribution of these measurements for each of the species.\n\nWith a scatter plot, we can select two of the features to plot on the x and y-a

In [4]:
s = 'Given the summary statistics and sample rows from the dataset, I would recommend a **scatter plot** for the exploratory data analysis in this instance.\n\n**Rationale:**\n\n1. **Summary Statistics:** The statistics reveal that the dataset consists of four continuous variables (sepal length, sepal width, petal length, petal width), each with a min, max, mean, and standard deviation (among others). Given these variables represent physical measurements, and knowing the target column seems associated with a category (e.g., "setosa", "versicolor", "virginica"), a scatter plot could be quite illuminating.\n\n2. **Sample Rows Observation:** The sample rows from the dataset include an additional column `"target"` that classifies the entries as species types ("setosa", "versicolor", "virginica"). A scatter plot would be an effective way to visualize the distribution of these measurements for each of the species.\n\nWith a scatter plot, we can select two of the features to plot on the x and y-axis and color each data point by the species type. This way, patterns such as clusters or separations among the species can be easily visualized and有助于发现不同种类之间的模式或分离。这种可视化可以帮助我们更直观地理解各种属性之间的关系，同时也能识别出物种之间的潜在差异。\n\n选择散点图可以从多个角度探索数据，如不同属性之间的关系、属性值的范围、每种花的分布情况等，这对于初步理解和分析这种多变量数据集非常有用。\n\n选择散点图中的“petal length”（花瓣长度）作为y轴和“petal width”（花瓣宽度）作为x轴，并用不同的颜色代表不同种类，是因为这四个样本行显示它们在“petal length”和“petal width”两关系具有较好的可分性，并且这两个属性可能是用于区分欧芹的最明显的特征之一（例如，可以很容易把setosa与versicolor和virginica区分开来）。\n\n因此，最终选择的回答应该是 \\boxed{scatter plot}。'
print(s)

Given the summary statistics and sample rows from the dataset, I would recommend a **scatter plot** for the exploratory data analysis in this instance.

**Rationale:**

1. **Summary Statistics:** The statistics reveal that the dataset consists of four continuous variables (sepal length, sepal width, petal length, petal width), each with a min, max, mean, and standard deviation (among others). Given these variables represent physical measurements, and knowing the target column seems associated with a category (e.g., "setosa", "versicolor", "virginica"), a scatter plot could be quite illuminating.

2. **Sample Rows Observation:** The sample rows from the dataset include an additional column `"target"` that classifies the entries as species types ("setosa", "versicolor", "virginica"). A scatter plot would be an effective way to visualize the distribution of these measurements for each of the species.

With a scatter plot, we can select two of the features to plot on the x and y-axis and c

## Diabetes

In [12]:
diabetes = load_diabetes()
df_diabetes = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

In [13]:
summary_statistics,sampled_rows = get_summary_stat_and_rows(df_diabetes)
prompt = get_prompt(summary_statistics,sampled_rows)

In [14]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [15]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #box plot

"Given the summary statistics and sample rows, I would recommend a Box plot as the best choice for visualization. \n\nThe rationale is as follows:\n- The summary statistics indicate that all variables are centered around zero or very close to it ('mean' column values are very close to zero, sometimes even ~0), which makes integer value labels for bar charts less appropriate here since it appears these are standardized or normalized values.\n- The data seems to be continuous in nature (with a 'count' and 'mean'), suggesting that histograms (for distribution exploration of individual variables) or box plots (for understanding spread, outliers, and comparison across variables) could be highly insightful.\n- Box plots are particularly useful here as they succinctly summarize the distribution of data for multiple variables across their quartiles, medians, ranges, and potential outliers, and they can be compared side by side effectively, which is not as easily done with, say, individual hist

In [3]:
s = "Given the summary statistics and sample rows, I would recommend a Box plot as the best choice for visualization. \n\nThe rationale is as follows:\n- The summary statistics indicate that all variables are centered around zero or very close to it ('mean' column values are very close to zero, sometimes even ~0), which makes integer value labels for bar charts less appropriate here since it appears these are standardized or normalized values.\n- The data seems to be continuous in nature (with a 'count' and 'mean'), suggesting that histograms (for distribution exploration of individual variables) or box plots (for understanding spread, outliers, and comparison across variables) could be highly insightful.\n- Box plots are particularly useful here as they succinctly summarize the distribution of data for multiple variables across their quartiles, medians, ranges, and potential outliers, and they can be compared side by side effectively, which is not as easily done with, say, individual histograms of each variable.\n- None of the variables exhibit obvious time-series properties (which would suggest a line chart) or nominal categories (which would suit a bar chart), but they do show variability which is well-addressed by a box plot.\n- Since there are numerous variables all centered on zero, a box plot can effectively communicate the variability within these already denormalized scores across all dimensions at once without needing confusing labeling or high-density plots.\n\nSo the recommended visualization is:\n\\boxed{Box plot}"
print(s)

Given the summary statistics and sample rows, I would recommend a Box plot as the best choice for visualization. 

The rationale is as follows:
- The summary statistics indicate that all variables are centered around zero or very close to it ('mean' column values are very close to zero, sometimes even ~0), which makes integer value labels for bar charts less appropriate here since it appears these are standardized or normalized values.
- The data seems to be continuous in nature (with a 'count' and 'mean'), suggesting that histograms (for distribution exploration of individual variables) or box plots (for understanding spread, outliers, and comparison across variables) could be highly insightful.
- Box plots are particularly useful here as they succinctly summarize the distribution of data for multiple variables across their quartiles, medians, ranges, and potential outliers, and they can be compared side by side effectively, which is not as easily done with, say, individual histograms

## Wine

In [16]:
wine = load_wine()
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['target'] = [wine.target_names[t] for t in wine.target]

In [17]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_wine)
prompt = get_prompt(summary_statistics,sampled_rows)
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [18]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #box plot

"Given the summary statistics and the sample rows, the dataset seems to contain measurements of various chemical components (such as alcohol content, flavanoids, proline) of wine, potentially for different types of wines labeled by `target`. There are numerical attributes like `alcohol`, `malic_acid`, etc., with a range and spread that can vary significantly, indicating they're likely continuous variables suitable for exploratory visualizations.\n\nA **box plot** would be an insightful visualization choice because it can effectively show the distribution of each numeric variable such as `alcohol`, `malic_acid`, `ash`, etc., revealing the central tendency, dispersion, and skewness. More importantly, if plotted separately for each value of the `target` variable (which seems to indicate different wine classes), the box plot can provide a direct comparison of how each variable's measurements differ across the different wine types. This will highlight differences that can be predictive (e.g

In [2]:
s = "Given the summary statistics and the sample rows, the dataset seems to contain measurements of various chemical components (such as alcohol content, flavanoids, proline) of wine, potentially for different types of wines labeled by `target`. There are numerical attributes like `alcohol`, `malic_acid`, etc., with a range and spread that can vary significantly, indicating they're likely continuous variables suitable for exploratory visualizations.\n\nA **box plot** would be an insightful visualization choice because it can effectively show the distribution of each numeric variable such as `alcohol`, `malic_acid`, `ash`, etc., revealing the central tendency, dispersion, and skewness. More importantly, if plotted separately for each value of the `target` variable (which seems to indicate different wine classes), the box plot can provide a direct comparison of how each variable's measurements differ across the different wine types. This will highlight differences that can be predictive (e.g., alcohol content for wine class 0 might have a higher median compared to class 2), aiding in pattern recognition.\n\nGiven the multi-dimensional and numerical nature of the data, and the need to easily compare distributions across the `target` classes, the box plot stands out as the most insightful starting point for exploratory data analysis.\n\n\\boxed{Box plot}"
print(s)

Given the summary statistics and the sample rows, the dataset seems to contain measurements of various chemical components (such as alcohol content, flavanoids, proline) of wine, potentially for different types of wines labeled by `target`. There are numerical attributes like `alcohol`, `malic_acid`, etc., with a range and spread that can vary significantly, indicating they're likely continuous variables suitable for exploratory visualizations.

A **box plot** would be an insightful visualization choice because it can effectively show the distribution of each numeric variable such as `alcohol`, `malic_acid`, `ash`, etc., revealing the central tendency, dispersion, and skewness. More importantly, if plotted separately for each value of the `target` variable (which seems to indicate different wine classes), the box plot can provide a direct comparison of how each variable's measurements differ across the different wine types. This will highlight differences that can be predictive (e.g., 

## Breast cancer

In [19]:
breast_cancer = load_breast_cancer()
df_breast_cancer = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df_breast_cancer['target'] = [breast_cancer.target_names[t] for t in breast_cancer.target]

In [20]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_breast_cancer)

In [21]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [22]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [23]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text # box plot

"Given the summary statistics and sample rows, we can observe that the dataset contains features which describe tumor characteristics (mean, worst, and error values for various shape-based factors). The target class indicates if the tumor is either benign or malignant.\n\nTo understand the distribution of these features or identify whether certain features differ notably between benign and malignant tumors, **box plots** for each feature could help in several ways:\n- Visualize the central tendency, spread and skewness of the data.\n- Easily spot any outliers.\n- Compare distributions between benign and malignant categories.\n\nGiven the number of features provided (e.g., mean_radius, mean_texture, etc.), box plots offer a concise way to summarize these distributions and provide insights into whether benign or malignant tumors have specific characteristics based on these features. Since these features are continuous and typically being analyzed to distinguish between two categories (be

In [1]:
s = "Given the summary statistics and sample rows, we can observe that the dataset contains features which describe tumor characteristics (mean, worst, and error values for various shape-based factors). The target class indicates if the tumor is either benign or malignant.\n\nTo understand the distribution of these features or identify whether certain features differ notably between benign and malignant tumors, **box plots** for each feature could help in several ways:\n- Visualize the central tendency, spread and skewness of the data.\n- Easily spot any outliers.\n- Compare distributions between benign and malignant categories.\n\nGiven the number of features provided (e.g., mean_radius, mean_texture, etc.), box plots offer a concise way to summarize these distributions and provide insights into whether benign or malignant tumors have specific characteristics based on these features. Since these features are continuous and typically being analyzed to distinguish between two categories (benign vs. malignant), a box plot would allow an easy qualitative visualization to grasp the feature's distribution breakdown according to the categories.\n\nTherefore, the best visualization in this context would be:\n\n\\boxed{Box plot}"
print(s)

Given the summary statistics and sample rows, we can observe that the dataset contains features which describe tumor characteristics (mean, worst, and error values for various shape-based factors). The target class indicates if the tumor is either benign or malignant.

To understand the distribution of these features or identify whether certain features differ notably between benign and malignant tumors, **box plots** for each feature could help in several ways:
- Visualize the central tendency, spread and skewness of the data.
- Easily spot any outliers.
- Compare distributions between benign and malignant categories.

Given the number of features provided (e.g., mean_radius, mean_texture, etc.), box plots offer a concise way to summarize these distributions and provide insights into whether benign or malignant tumors have specific characteristics based on these features. Since these features are continuous and typically being analyzed to distinguish between two categories (benign vs.